# Sanity check

In [2]:
!pwd

/home/amar/Desktop/projects/nfs_object_detection


In [2]:
! which python

/home/amar/Desktop/projects/nfs_object_detection/env/bin/python


In [7]:
for i in range(1000):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# Imports

In [3]:
import os
import cv2
import numpy as np
import shutil
import random

# Paths and files

In [19]:
project_dir = os.getcwd()
paths = {
    'raw_video': os.path.join(os.getcwd(),'data','raw','videos'),
    'raw_image' : os.path.join(os.getcwd(),'data','raw','images'),
    'processed_video': os.path.join(os.getcwd(),'data','processed','videos'),
    'processed_image' : os.path.join(os.getcwd(),'data','processed','images'),
    'yolov3_tiny_config': os.path.join(project_dir, 'my_models', 'yolov3_tiny', 'yolov3_tiny.cfg'), 
    'yolov3_tiny_weights': os.path.join(project_dir, 'my_models', 'yolov3_tiny', 'yolov3_tiny.weights'),
    'yolov3_tiny_model': os.path.join(project_dir, 'my_models', 'yolov3_tiny', 'yolov3_tiny.h5'),
    'yolov3_320_config': os.path.join(project_dir, 'my_models', 'yolov3_320', 'yolov3_320.cfg'), 
    'yolov3_320_weights': os.path.join(project_dir, 'my_models', 'yolov3_320', 'yolov3_320.weights'),
    'yolov3_320_model': os.path.join(project_dir, 'my_models', 'yolov3_320', 'yolov3_320.h5'),
    'labels_dir': os.path.join(project_dir, 'data', 'labels'),
    'images_dir': os.path.join(project_dir, 'data', 'images'),
    'final_imgs_labels_dir': os.path.join(project_dir, 'colab', 'data', 'obj'),
    'colab_train.txt': os.path.join(project_dir, 'colab', 'data', 'train.txt'),
    'colab_test.txt': os.path.join(project_dir, 'colab', 'data', 'test.txt')
}

# Video to image

In [22]:
input_video_path = os.path.join(paths['raw_video'], 'video3.avi')
output_image_dir = os.path.join(paths['raw_image'])

In [23]:
#* Save the frames needed in the set wanted_frames
cap = cv2.VideoCapture(input_video_path)
count = 0
total_frames = cap.get(7)
# print(total_frames)
wanted_frames = set()
try:
   while True:
      ret, frame = cap.read()
      cv2.imshow('Input Video', frame)
      if cv2.waitKey(0) == ord('k'):
         wanted_frames.add(count)
      elif cv2.waitKey(0) == ord('q'):
         break
      count += 1
except:
   # print(count)
   print('Video not read, exiting..')

cv2.destroyAllWindows()
cap.release()
# print('wanted_frames:', wanted_frames)


Video not read, exiting..


In [33]:
print(len(wanted_frames))

8412


In [32]:
#* Save every 20th frame from the wanted_frames set
cap = cv2.VideoCapture(input_video_path)
count = 0
kept = 0
try:
    while True:
        ret, frame = cap.read()
        cv2.imshow('Input Video', frame)
        if count in wanted_frames:
            kept += 1
            if kept % 20 == 0:
                cv2.imwrite(os.path.join(output_image_dir, str(count) + '.jpg'), frame)
        if cv2.waitKey(10) == ord('q'):
            break
        count += 1
except Exception as e:
    cv2.destroyAllWindows()
    print(e)

# Colab Data preparation

In [8]:
paths

{'raw_video': '/home/amar/Desktop/projects/nfs_object_detection/data/raw/videos',
 'raw_image': '/home/amar/Desktop/projects/nfs_object_detection/data/raw/images',
 'processed_video': '/home/amar/Desktop/projects/nfs_object_detection/data/processed/videos',
 'processed_image': '/home/amar/Desktop/projects/nfs_object_detection/data/processed/images',
 'yolov3_tiny_config': '/home/amar/Desktop/projects/nfs_object_detection/my_models/yolov3_tiny/yolov3_tiny.cfg',
 'yolov3_tiny_weights': '/home/amar/Desktop/projects/nfs_object_detection/my_models/yolov3_tiny/yolov3_tiny.weights',
 'yolov3_tiny_model': '/home/amar/Desktop/projects/nfs_object_detection/my_models/yolov3_tiny/yolov3_tiny.h5',
 'yolov3_320_config': '/home/amar/Desktop/projects/nfs_object_detection/my_models/yolov3_320/yolov3_320.cfg',
 'yolov3_320_weights': '/home/amar/Desktop/projects/nfs_object_detection/my_models/yolov3_320/yolov3_320.weights',
 'yolov3_320_model': '/home/amar/Desktop/projects/nfs_object_detection/my_models/

In [11]:
# Storing the labels and images in a list to see which of the images were not labelled
labels = list()
images = list()

for dirpath, dirnames, filenames in os.walk(paths['labels_dir']):
    for file in filenames:
        labels.append(file[0:-4])

for dirpath, dirnames, filenames in os.walk(paths['images_dir']):
    for file in filenames:
        images.append(file[0:-4])

labels = set(labels)
images = set(images)

# Find the ones that are common
images_and_labels = labels.intersection(images)
len(images_and_labels)

335

## colab/data/obj folder

In [25]:
for i, image_id in enumerate(images_and_labels):
    img_src_path = os.path.join(paths['images_dir'], image_id + '.jpg')
    label_src_path = os.path.join(paths['labels_dir'], image_id + '.txt')
    dst_path = os.path.join(project_dir, 'colab', 'data', 'obj')
    shutil.copy(img_src_path, dst_path)
    shutil.copy(label_src_path, dst_path)
    print('{} files done'.format(i))

0 files done
1 files done
2 files done
3 files done
4 files done
5 files done
6 files done
7 files done
8 files done
9 files done
10 files done
11 files done
12 files done
13 files done
14 files done
15 files done
16 files done
17 files done
18 files done
19 files done
20 files done
21 files done
22 files done
23 files done
24 files done
25 files done
26 files done
27 files done
28 files done
29 files done
30 files done
31 files done
32 files done
33 files done
34 files done
35 files done
36 files done
37 files done
38 files done
39 files done
40 files done
41 files done
42 files done
43 files done
44 files done
45 files done
46 files done
47 files done
48 files done
49 files done
50 files done
51 files done
52 files done
53 files done
54 files done
55 files done
56 files done
57 files done
58 files done
59 files done
60 files done
61 files done
62 files done
63 files done
64 files done
65 files done
66 files done
67 files done
68 files done
69 files done
70 files done
71 files done
72

In [27]:
for dirpath, dirnames, filenames in os.walk(paths['final_imgs_labels_dir']):
    print(dirpath, len(dirnames), len(filenames))

/home/amar/Desktop/projects/nfs_object_detection/colab/data/obj 0 670


## Train and Test .txt files

In [14]:
len(images_and_labels)

335

In [18]:
test = set(random.sample(images_and_labels, 40))
train = images_and_labels - test
len(test), len(train), len(test)/(len(test) + len(train))

(40, 295, 0.11940298507462686)

In [22]:
with open(paths['colab_test.txt'], 'w') as f:
    for test_image in test:
        f.write(os.path.join('data/obj/' + test_image + '.jpg' + '\n'))

In [23]:
with open(paths['colab_train.txt'], 'w') as f:
    for train_image in train:
        f.write(os.path.join('data/obj/' + train_image + '.jpg' + '\n'))

## Models